In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
import nltk

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

import plotly 
import plotly.offline as offline
import plotly.graph_objs as go
from collections import Counter

import os
from scipy import sparse
from scipy.sparse import csr_matrix

from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import random
from wordcloud import WordCloud
import nltk
from tqdm import tqdm
warnings.filterwarnings("ignore")

#from mlxtend.classifier import StackingClassifier

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

In [ ]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
data=pd.read_csv('../input/inputdata/train.tsv', sep='\t')
x_test=pd.read_csv('../input/test-data/test_stg2.tsv', sep='\t')
data.head()


In [ ]:
x_test.head()

In [ ]:
submission_final= pd.DataFrame(x_test['test_id'])
submission_final.head(5)

In [ ]:
data.info()

In [ ]:
data.describe()

**Checking if any column has missing data**

In [ ]:
data.isnull().any()
#df_train.brand_name.isnull().sum()
x_test.isnull().any()

We will be dropping rows that have price <3 as it is said that mercari doesn't allow products to be soled for <3 and so this must be an error

In [ ]:
data = data.drop(data[(data.price < 3.0)].index)


In [ ]:
data.shape

**Splitting category feature**

In [ ]:
def transform_category_name(category_name):
    try:
        main, sub1, sub2= category_name.split('/')
        return main, sub1, sub2
    except:
        return ('missing', 'missing', 'missing')

data['category_main'], data['category_sub1'], data['category_sub2'] = zip(*data['category_name'].apply(transform_category_name))
x_test['category_main'], x_test['category_sub1'], x_test['category_sub2'] = zip(*x_test['category_name'].apply(transform_category_name))

data.head()

**Fill the missing values as below**

In [ ]:
def fill_nan(data):
  '''
  Function to fill the NaN values in various columns
  '''
  data["item_description"].fillna("No description yet",inplace=True)
  data["brand_name"].fillna("missing",inplace=True)
  data["category_name"].fillna("missing",inplace=True)
  return data

In [ ]:
names = list(data['name'].values)

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
df1=data[data['brand_name'].isna()]


In [ ]:
df1.head()

In [ ]:
len(df1)

In [ ]:
data.head()
data.brand_name.isnull().sum()


In [ ]:
known_brands = data['brand_name'].unique()
print(known_brands)
known_brands = np.delete(known_brands, [0])
print(known_brands)

In [ ]:
x_train= fill_nan(data)
x_train.head()  
x_test=fill_nan(x_test)
x_test=x_test.drop(columns=['category_name'])

x_test

In [ ]:
#x=data.drop(columns=['price','category_name'])
x_train.head()


**Price distribution**

In [ ]:
data["logPrice"] = np.log(data['price']+1)
data.head()

In [ ]:
y_train=data.logPrice
y=data.price


In [ ]:
plt.figure(figsize=(10,3))
sns.distplot(y_train, hist=True, label="Price")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))

# Histogram of the Price
plt.style.use('fivethirtyeight')
plt.hist(y, bins = 100, edgecolor = 'k');
plt.xlabel('Price'); plt.ylabel('Number of items'); 
plt.title('Price Distribution');

In [ ]:
#Box-plot using log(price+1)
fig, ax = plt.subplots(figsize=(14,5))
plt.title('Price distribution', fontsize=15)
sns.boxplot(data.logPrice,showfliers=False)
ax.set_xlabel('log(Price+1)',fontsize=15)

plt.show()


We have scaled down our 'price' feature logprice. We have added log(Price+1) to it as log(0) is undefined. hence if price for an item is 0, then the item will have no price as defined.

In [ ]:
#Histogram using log(price+1)
fig, ax = plt.subplots(figsize=(14,8))
ax.hist(data.logPrice,bins=30,range=[0,8],label="Price")
plt.title('log Price distribution', fontsize=15)
ax.set_xlabel('log(Price+1)',fontsize=15)
ax.set_ylabel('No of items',fontsize=15)

plt.show()

The 'log(price+1)' feature has the price range spread over 2.3-3.5 for most of the items

**Binning Price Into Two Categories**

In [ ]:
bins = [0, 64, 5000]
labels = ['cheap','expensive']
data['price_level'] = pd.cut(data['price'], bins=bins, labels=labels)

data.head()

**shipping distribution:**

In [ ]:
plt.figure(figsize=(6, 4))

# Histogram of the Price
plt.style.use('fivethirtyeight')
plt.hist(x_train['shipping'].values, bins = 100, edgecolor = 'k');
plt.xlabel('Price'); plt.ylabel('Number of items'); 
plt.title('Price Distribution');

In [ ]:
data.groupby('shipping')['price'].describe()


In [ ]:
shipping = data[data['shipping']==1]['price']
no_shipping = data[data['shipping']==0]['price']

plt.figure(figsize=(12,7))
plt.hist(shipping, bins=50, range=[0,250], alpha=0.7, label='Price With Shipping')
plt.hist(no_shipping, bins=50, range=[0,250], alpha=0.7, label='Price With No Shipping')
plt.title('Price Distrubtion With/Without Shipping', fontsize=15)
plt.xlabel('Price')
plt.ylabel('Normalized Samples')
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.legend(fontsize=15)
plt.show()

In [ ]:
#ref - https://machinelearningmastery.com/parametric-statistical-significance-tests-in-python/
''' This code conducts a statistical significance test 
using One way ANOVA to see if prices in shipping differ significantly '''

from scipy.stats import f_oneway

stat, p = f_oneway(x_train.price.values[x_train.shipping == 1], x_train.price.values[x_train.shipping == 0])
print('Statistics=%.9f, p=%.9f' % (stat, p))

# interpret
alpha = 0.05
if p > alpha:
    print('Same distributions (fail to reject H0)')
else:
    print('Different distributions (reject H0)')


**Statistics summary** - Clearly, ANOVA test suggests that there is a significant difference and 'p' value is quite negligible - so we will be going forward with our conclusion that there is indeed a significant enough difference between the 2 shipping categories in the price values


**conclusion:**
>>We can conclude that lower prices hhave shipping paid by buyers and higher prices by sellers.


**category distribution:**

In [ ]:
print("No of unique values in item category is:",data['category_name'].nunique())
print("No of unique values in item category is:",data['category_name'].value_counts()[:10])


In [ ]:
print("No of unique values in main category: ",x_train['category_main'].nunique())
print("No of unique values in Sub_category1: ",x_train['category_sub1'].nunique())
print("No of unique values in Sub_category2: ",x_train['category_sub2'].nunique())

In [ ]:

#Ref: https://seaborn.pydata.org/generated/seaborn.countplot.html
fig, ax = plt.subplots(figsize=(13,6))
sns.countplot(x='category_main', data=x_train, ax=ax)
plt.title('Item count by main_category',fontsize=25)
plt.ylabel('No of items',fontsize=25)
plt.xlabel('')
plt.xticks(rotation=70,fontsize=20)
plt.yticks(fontsize=20)

plt.show()

In [ ]:
#main = pd.DataFrame(cat_train['category_main'].value_counts()).reset_index().rename(columns={'index': 'main', 'category_main':'count'})
fig, axes = plt.subplots(figsize=(12, 5))
main = data[data['price']<100]
# Use a color palette
ax = sns.boxplot( x=main["category_main"], y=main["price"], palette="Blues")
ax.set_xticklabels(ax.get_xticklabels(),rotation=90, fontsize=12)



In [ ]:
#Ref: https://www.kaggle.com/konohayui/mercari-price-suggestion-eda
wc = WordCloud(max_words=300,width = 1200, height = 900).generate(" ".join(x_train['category_main']))
plt.figure(figsize = (18, 8))
plt.imshow(wc)
plt.axis("off")
plt.show()

In [ ]:
#Ref: https://stackoverflow.com/questions/48799185/plot-histogram-matplotlib-with-labels-on-x-axis-instead-of-count
print("Top-10 subcategory_1:")
x_train.category_sub1.value_counts()[:10].plot(kind = 'bar',figsize = (13,5), title="Top_10 subcategory_1",fontsize=20)

In [ ]:
#Ref: https://www.kaggle.com/konohayui/mercari-price-suggestion-eda
wc = WordCloud(max_words=300,width = 1200, height = 900).generate(" ".join(x_train['category_sub1']))
plt.figure(figsize = (18, 8))
plt.imshow(wc)
plt.axis("off")
plt.show()

In [ ]:
#Ref: https://stackoverflow.com/questions/48799185/plot-histogram-matplotlib-with-labels-on-x-axis-instead-of-count
print("Top-10 subcategory_2:")
x_train.category_sub2.value_counts()[:10].plot(kind = 'bar',figsize = (13,5), title="Top_10 subcategory_2",fontsize=20)

In [ ]:
#Ref: https://www.kaggle.com/konohayui/mercari-price-suggestion-eda
wc = WordCloud(max_words=300,width = 1200, height = 900).generate(" ".join(x_train['category_sub2']))
plt.figure(figsize = (18, 8))
plt.imshow(wc)
plt.axis("off")
plt.show()

**CONCLUSION:**
>> We can conclude that Women category is the highest, followed by clothing.

**Brand name distribution**

In [ ]:
print("No of unique values in brand are:",x_train['brand_name'].nunique())
print("No of unique values in brand are is:",x_train['brand_name'].value_counts())
x_train.brand_name.value_counts()[:10].plot(kind = 'bar',figsize = (13,5), title="Brand",fontsize=20)

# Display Top 20 Expensive Brands By Mean Price


In [ ]:
# Display Top 20 Expensive Brands By Mean Price
top20_brand = data.groupby('brand_name', axis=0).mean()
df_expPrice = pd.DataFrame(top20_brand.sort_values('price', ascending = False)['price'][0:20].reset_index())

ax = sns.barplot(x="brand_name", y="price", data=df_expPrice)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90, fontsize=15)
ax.set_title('Top 20 Expensive Brand', fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.show()# Displayd frequency ratio of brand names


**CONCLUSION**
>> We could see most of the brand names are missing .

**Item description:**


Word plot gives most frequent words used in the field

In [ ]:

#Ref: https://www.kaggle.com/konohayui/mercari-price-suggestion-eda
wc = WordCloud(max_words=300,width = 1200, height = 900).generate(" ".join(x_train.item_description.astype(str)))
plt.figure(figsize = (18, 8))
plt.imshow(wc)
plt.axis("off")
plt.show()

**TEXT PRE-PROCESSING AND FEATURIZATION**




# BRAND NAME

In [ ]:
#this function assigns a value 1 if a product has brand_name else 0"""

def branded(data):
   is_branded=[]
   for i in data['brand_name']:
      if i=='missing': 
          is_branded.append(0) 
      else: 
          is_branded.append(1)
   return is_branded
x_train['is_branded']=branded(x_train)
x_test['is_branded']=branded(x_test)

In [ ]:
""" 
    This function is used to guess the missing brand name.
    It will check for an existing brand name mentioned in the item name section and also category of brnad name and populates the missing brand
"""
def fill_brand(row):
    brand, name, cn = row
    brand = brand.lower()
    if brand=='missing':
        for brand in known_brands:
            if str(brand) in name:
                return brand         
    return brand
x_train['fillmissingbrands'] = x_train[['brand_name','name', 'category_main']].apply(fill_brand, axis = 1)
x_test['fillmissingbrands'] = x_test[['brand_name','name', 'category_main']].apply(fill_brand, axis = 1)


In [ ]:
# https://stackoverflow.com/a/47091490/4084039
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [ ]:
# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"]

In [ ]:
# Combining all the above stundents 
from tqdm import tqdm
def preprocess_text(text_data):
    preprocessed_text = []
    # tqdm is for printing the status bar
    for sentance in tqdm(text_data):
        sent = decontracted(sentance)
        sent = sent.replace('\\r', ' ')
        sent = sent.replace('\\n', ' ')
        sent = sent.replace('\\"', ' ')
        sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
        # https://gist.github.com/sebleier/554280
        sent = ' '.join(e for e in sent.split() if e.lower() not in stopwords)
        preprocessed_text.append(sent.lower().strip())
    return preprocessed_text

In [ ]:
preprocessed_item_description = preprocess_text(x_train['item_description'].values)
print(preprocessed_item_description[:5])
preprocessed_test_item_description = preprocess_text(x_test['item_description'].values)

In [ ]:
preprocessed_test_item_description[1]

In [ ]:
word_punctuation_tokenizer = nltk.WordPunctTokenizer()
word_tokenized_corpus = [word_punctuation_tokenizer.tokenize(sent) for sent in preprocessed_test_item_description]


In [ ]:
#How to calculate number of words in a string in DataFrame: https://stackoverflow.com/a/37483537/4084039

x_train['preprocessed_item_description'] = preprocessed_item_description
x_test['preprocessed_item_description'] = preprocessed_test_item_description

word_count = x_train['preprocessed_item_description'].str.split().apply(len).value_counts()
word_dict = dict(word_count)
word_dict = dict(sorted(word_dict.items(), key=lambda kv: kv[1])[0:10])
x_train['count'] = x_train['preprocessed_item_description'].apply(lambda x : len(str(x)))
x_test['count'] = x_test['preprocessed_item_description'].apply(lambda x : len(str(x)))



**Price Vs Description length**


In [ ]:
plt.scatter(data['count'],data['price'],alpha=0.8)
plt.title("Price Vs Description length")
plt.xlabel("Description length")
plt.ylabel("Price(USD)")
plt.show()

Here, description length vs price has been given. From the plot, it can be said as length increases, price charged becomes lesser and lesser. Most of the items with lesser description length have more price value.

**Calculating sentiment score.**

In [ ]:

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')


In [ ]:
#performing sentiment analysis 

def sentiment_analysis(data):
   sentiment_score = SentimentIntensityAnalyzer()
   sentiment = []
   for sentence in tqdm(data):
       sentiment.append(sentiment_score.polarity_scores(sentence))
   return sentiment


training_sentiment_score=sentiment_analysis(x_train['item_description']) 
cv_sentiment_score=sentiment_analysis(x_test['item_description'])


In [ ]:
#this function splits sentiment analysis score into four further features ie positive,negative,compound and neutral

def splitting_sentiment(sentiment_score):
  positive=[]
  negative=[]
  neutral=[]
  compound=[]
  for i in tqdm(sentiment_score):
    positive.append(i['pos'])
    negative.append(i['neg'])
    neutral.append(i['neu'])
    compound.append(i['compound'])
  return positive,negative,neutral,compound

In [ ]:
#Training Data Sentiment Analysis
pos,neg,neu,comp=splitting_sentiment(training_sentiment_score)
x_train['positive']=pos
x_train['negative']=neg
x_train['neutral']=neu
x_train['compound']=comp


In [ ]:

#x_test Data Sentiment Analysis
pos,neg,neu,comp=splitting_sentiment(cv_sentiment_score)
x_test['positive']=pos
x_test['negative']=neg
x_test['neutral']=neu
x_test['compound']=comp


In [ ]:
x=x_train.drop(columns=['item_description'])
x_train.head()

**Item condition** **distribution** 


In [ ]:
sns.set(rc={'figure.figsize':(8,7)})
sns.ax = sns.boxplot(x=x_train["item_condition_id"], y=np.log(x_train['price']+1), palette="Blues")
plt.title('Violin Plots - Price variation in item_condition_id')
plt.show()

In [ ]:
#ref - https://machinelearningmastery.com/parametric-statistical-significance-tests-in-python/
# applying statistical significance test to confirm if the price values are not by chance and are according to item condition.
from scipy.stats import f_oneway
stat, p = f_oneway(x_train.price.values[x_train.item_condition_id == 1],
                   x_train.price.values[x_train.item_condition_id == 2],
                   x_train.price.values[x_train.item_condition_id == 3],
                   x_train.price.values[x_train.item_condition_id == 4],
                   x_train.price.values[x_train.item_condition_id == 5])
print('Statistics=%.9f, p=%.9f' % (stat, p))

alpha = 0.05
if p > alpha:
    print('Same distributions (fail to reject H0)')
else:
    print('Different distributions (reject H0)')

**conclusion:**
1. we can say that the distrinbution of prices among different conditions are significantly different.

**Name Feature:**

In [ ]:
preprocessed_name = preprocess_text(x_train['name'].values)
print(preprocessed_name[:5])
x_train['name'] = preprocessed_name
test_preprocessed_name=preprocess_text(x_test['name'].values)
x_test['name']=test_preprocessed_name

**Correlation between features**

In [ ]:
# gives correlation with the price and sort 
correlations_data = x_train.corr()['price'].sort_values()
print(correlations_data)

In [ ]:
columns = list(x_train.columns)
plt.figure(figsize = (10, 10))
sns.heatmap(x_train[columns].corr(), annot = True, linewidth = 0.5)
plt.show()

**CONCLUSION:**
1. We can conclude that all of the features except train-id has some significant effect on the price  including sentiment scores.
2. We can tell that for shipping it is negatively related which states that lower prices requires buyers to pay and higher prices are paid mostly by seller.
3. Also, we could see the description length (count) is positively linear so as the length of the description  increases price seems to be higher assuming mostly it would be a positive description.





**split train data into train and cv data**

In [ ]:
x_train.columns

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_cv, y_tr, y_cv = train_test_split(x_train, y_train, test_size=0.20)


# **Feature Engineering:**

**CATEGORICAL FEATURES:**

**Name feature**

In [ ]:
vectorizer = CountVectorizer(binary=True)
vectorizer.fit(x_train['name'].values) 

train_name = vectorizer.transform(x_train['name'].values)
cv_name = vectorizer.transform(x_cv['name'].values)
test_name = vectorizer.transform(x_test['name'].values)

**Main category**

In [ ]:
vectorizer = CountVectorizer(binary=True)
vectorizer.fit(x_train['category_main'].values) 

train_category_main = vectorizer.transform(x_train['category_main'].values)
cv_category_main = vectorizer.transform(x_cv['category_main'].values)
test_category_main = vectorizer.transform(x_test['category_main'].values)

**Sub Category 1**

In [ ]:
vectorizer = CountVectorizer(binary=True)
vectorizer.fit(x_train['category_sub1'].values) 
train_category_sub1 = vectorizer.transform(x_train['category_sub1'].values)
cv_category_sub1 = vectorizer.transform(x_cv['category_sub1'].values)
test_category_sub1 = vectorizer.transform(x_test['category_sub1'].values)


**Sub Category 2**

In [ ]:
vectorizer = CountVectorizer(binary=True)
vectorizer.fit(x_train['category_sub2'].values)

train_category_sub2 = vectorizer.transform(x_train['category_sub2'].values)
cv_category_sub2 = vectorizer.transform(x_cv['category_sub2'].values)
test_category_sub2 = vectorizer.transform(x_test['category_sub2'].values)


**Brand Name**

vectorizer = CountVectorizer(binary=True)
vectorizer.fit(x_train['brand_name'].values) # fit has to happen only on train data

# we use the fitted CountVectorizer to convert the text to vector
train_brand_name = vectorizer.transform(x_train['brand_name'].values)
train_brand_name = vectorizer.transform(x_test['brand_name'].values)

print("After vectorizations")
print(train_brand_name.shape)

print(vectorizer.get_feature_names())
print("="*100)

In [ ]:
vectorizer = CountVectorizer(binary=True) 
vectorizer.fit(x_train['brand_name'].values) 

train_brand_name = vectorizer.transform(x_train['brand_name'].values) 
cv_brand_name = vectorizer.transform(x_cv['brand_name'].values) 
test_brand_name = vectorizer.transform(x_test['brand_name'].values)

**Item condition**

[link text](https://)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

oe_style = OneHotEncoder()
train_condition = oe_style.fit_transform(x_train[["item_condition_id"]])
cv_condition = oe_style.transform(x_cv[["item_condition_id"]])
test_condition = oe_style.transform(x_test[["item_condition_id"]])

pd.DataFrame(train_condition.toarray(), columns=oe_style.categories_).head()


**Shipping**

In [ ]:
oe_style = OneHotEncoder()
train_shipping = oe_style.fit_transform(x_train[["shipping"]])
cv_shipping = oe_style.transform(x_cv[["shipping"]])
test_shipping = oe_style.transform(x_test[["shipping"]])

pd.DataFrame(train_shipping.toarray(), columns=oe_style.categories_).head()

**Is Branded**

In [ ]:
oe_style = OneHotEncoder()
train_is_branded = oe_style.fit_transform(x_train[["is_branded"]])
cv_is_branded = oe_style.transform(x_cv[["is_branded"]])
test_is_branded = oe_style.transform(x_test[["is_branded"]])

pd.DataFrame(train_is_branded.toarray(), columns=oe_style.categories_).head()

# ***Text Features***

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=10)
vectorizer.fit(x_train['preprocessed_item_description'])
X_train_itemdes = vectorizer.transform(x_train['preprocessed_item_description'])
X_cv_itemdes = vectorizer.transform(x_cv['preprocessed_item_description'])
X_test_itemdes = vectorizer.transform(x_test['preprocessed_item_description'])


**Numerical features**

In [ ]:
from sklearn.preprocessing import MinMaxScaler 
normalizer = MinMaxScaler(feature_range=(-1, 1))

normalizer.fit(x_train['negative'].values.reshape(-1,1))

X_train_neg_norm = normalizer.transform(x_train['negative'].values.reshape(-1,1))
X_cv_neg_norm = normalizer.transform(x_cv['negative'].values.reshape(-1,1))
X_test_neg_norm = normalizer.transform(x_test['negative'].values.reshape(-1,1))

In [ ]:
normalizer = MinMaxScaler(feature_range=(-1, 1))
normalizer.fit(x_train['neutral'].values.reshape(-1,1))

X_train_neu_norm = normalizer.transform(x_train['neutral'].values.reshape(-1,1))
X_cv_neu_norm = normalizer.transform(x_cv['neutral'].values.reshape(-1,1)) 
X_test_neu_norm = normalizer.transform(x_test['neutral'].values.reshape(-1,1))


In [ ]:
normalizer = MinMaxScaler(feature_range=(-1, 1))

normalizer.fit(x_train['positive'].values.reshape(-1,1))

X_train_pos_norm = normalizer.transform(x_train['positive'].values.reshape(-1,1)) 
X_cv_pos_norm = normalizer.transform(x_cv['positive'].values.reshape(-1,1)) 
X_test_pos_norm = normalizer.transform(x_test['positive'].values.reshape(-1,1))

In [ ]:
normalizer =MinMaxScaler(feature_range=(-1, 1))

normalizer.fit(x_train['compound'].values.reshape(-1,1))

X_train_com_norm = normalizer.transform(x_train['compound'].values.reshape(-1,1))
X_cv_com_norm = normalizer.transform(x_cv['compound'].values.reshape(-1,1))
X_test_com_norm = normalizer.transform(x_test['compound'].values.reshape(-1,1))

In [ ]:
normalizer = MinMaxScaler(feature_range=(-1, 1))

normalizer.fit(x_train['count'].values.reshape(-1,1))

X_train_count = normalizer.transform(x_train['count'].values.reshape(-1,1))
X_cv_count = normalizer.transform(x_cv['count'].values.reshape(-1,1))
X_test_count = normalizer.transform(x_test['count'].values.reshape(-1,1))

In [ ]:
normalizer = MinMaxScaler(feature_range=(-1, 1))

normalizer.fit(y_tr.values.reshape(-1,1))

X_train_price_norm = normalizer.transform(y_tr.values.reshape(-1,1))
X_cv_price_norm = normalizer.transform(y_cv.values.reshape(-1,1))


**Merger sparse matrices**

In [ ]:
from scipy.sparse import hstack
X_tr = hstack((train_name,train_category_main,train_category_sub1,train_category_sub2, train_brand_name, train_condition, train_shipping,X_train_itemdes,X_train_count,X_train_neg_norm,X_train_neu_norm,X_train_pos_norm,X_train_com_norm,train_is_branded)).tocsr()
X_cv = hstack((cv_name,cv_category_main,cv_category_sub1,cv_category_sub2, cv_brand_name, cv_condition, cv_shipping,X_cv_itemdes,X_cv_count,X_cv_neg_norm,X_cv_neu_norm,X_cv_pos_norm,X_cv_com_norm,cv_is_branded)).tocsr()
X_te = hstack((test_name,test_category_main,test_category_sub1,test_category_sub2, test_brand_name, test_condition, test_shipping,X_test_itemdes,X_test_count,X_test_neg_norm,X_test_neu_norm,X_test_pos_norm,X_test_com_norm,test_is_branded)).tocsr()


**ML MODELLING**

**RIDGE REGRESSOR**
https://www.statisticshowto.com/ridge-regression/#:~:text=Ridge%20regression%20is%20a%20way,(correlations%20between%20predictor%20variables).

https://towardsdatascience.com/ridge-regression-for-better-usage-2f19b3a202db

In [ ]:
from sklearn.linear_model import SGDRegressor, Ridge


In [ ]:
params = {"alpha":[0.000001,0.00001,0.0001,0.001,0.01,0.1,0,1,10,100,1000,10000,100000]}
ridge_model = Ridge(solver = "lsqr", fit_intercept=False)
lr_reg = RandomizedSearchCV(ridge_model,param_distributions =params,n_jobs=-1,random_state=0)
lr_reg.fit(X_tr, y_tr)
lr_reg.best_params_

In [ ]:
ridge_model2 = Ridge(alpha=0.01,solver = "lsqr", fit_intercept=False )
ridge_model2.fit(X_tr, y_tr)
y_pred = ridge_model2.predict(X_cv)
ridge_RMSLE = np.sqrt(mean_squared_error(y_cv, y_pred))
print(ridge_RMSLE)

In [ ]:
ridge_model2 = Ridge(alpha=0.01,solver = "lsqr", fit_intercept=False )
ridge_model2.fit(X_tr, y_tr)

pred_ridge = ridge_model2.predict(X_te)
print(pred_ridge)


In [ ]:
submission_final['price'] = np.expm1(pred_ridge)
submission_final.to_csv("submission.csv", index=False)